Seq2Seq(Sequence-to-Sequence)
* 순서
[원문 문장] → [Encoder] → [Context Vector] → [Decoder] → [출력 문장]

<span style="color: Gold"> 1. 학습용 영어-프랑스어 병렬 문장 데이터 준비

개념:   
입력(영어)과 출력(프랑스어) 쌍으로 구성  
디코더 입력에는 시작 토큰(\t), 타겟에는 종료 토큰(\n) 추가  

설명:  
input_texts: 인코더에 입력될 영어 문장  
target_texts: 디코더가 생성해야 할 프랑스어 문장 (전처리 포함)  

In [3]:
import numpy as np
import tensorflow as tf

data_pairs = [
    ("Hello", "Bonjour"),
    ("How are you", "Comment allez-vous"),
    ("Good morning", "Bonjour matin"),
    ("Thank you", "Merci"),
]

# 입력과 타겟을 분리
input_texts = []
target_texts = []
for eng,fra in data_pairs:
    input_texts.append(eng)
    # 디코더 입력 '\t'(시작), 디코더 출력: '\n' (종료)
    target_texts.append(f'\t{fra}\n')
for i in range(len(input_texts)):
    print(f'입력: {input_texts[i]:20s}--> 타겟:{target_texts[i]}')

입력: Hello               --> 타겟:	Bonjour

입력: How are you         --> 타겟:	Comment allez-vous

입력: Good morning        --> 타겟:	Bonjour matin

입력: Thank you           --> 타겟:	Merci



---

<span style="color: Gold"> 2. 문자 단위 사전(vocabulary) 생성 및 정수 인덱스 변환  

 개념:  
각 문자를 고유한 정수로 매핑  
입력과 타겟의 사전은 별도 관리   
원-핫 인코딩으로 신경망 입력 형태 생성  

설명:  
input_characters: 영어 문장에 등장하는 모든 고유 문자  
target_characters: 프랑스어 문장 + 특수 토큰(\t, \n)  
encoder_input_data: 3D 배열 (샘플, 시퀀스 길이, 문자 사전 크기)  

문자를 숫자 인덱스로 바꾸기 위한 매핑 테이블을 만드는 것  
-> 문자열 데이터 → 문자 집합 추출 → 인덱스 부여 → 인코딩 준비  

In [30]:
# 입력과 타겟의 고유한 문자 수집
# input_characters = set()
# target_characters = set()

# for text in input_texts:
#     for char in text:
#         input_characters.add(char)

# 문자 집합 만들기
# 단어 하나하나를 집합으로 만드는 것
# 입력 문자 집합 (input_characters) = {H, e, l, o, w}
# 출력 문자 집합 (target_characters) = {B, o, n, j, u, r, S, a, l, t}
# { }은 set이 자동으로 실행되어 중복은 제거된 상태
input_characters = {char for text in input_texts for char in text }
target_characters = {char for target_text in target_texts for char in target_text}

# 정렬해서 일관성 확보
# 각각 집합 내부에서 일관성 확보
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

# 토큰(문자)개수 계산
# 집합의 길이에 따라 원핫 벡터 차원을 결정하기 위해서. 문자 집합이 5개면 원핫 벡터 길이 5차원이 필요
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

# 가장 긴 문장 길이 계산
# 시퀀스 길이가 달라도, 뱅ㄹ 크기는 고정해서 LSTM에 넣을 수 잇음
# 짧은 문장은 0-padding을 사용
# 모든 문장을 같은 길이로 맞춤
# 가장 긴 문장 길이에 맞춰 패딩 기준을 맞추기 위해 하는 작업
max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

# 문자 -> 인덱스 매핑
# 문자의 순서대로 인덱스를 부여하여 문자와 그 번호가 매칭되도록 딕셔너리를 만들어줌
# 문자: 인덱스 숫자번호 -> 딕셔너리 형태로 만듦
input_token_index = { char:i for i,char in enumerate(input_characters)}
target_token_index = { char:i for i,char in enumerate(target_characters)}

# 인덱스 -> 문자로 역매핑(추론시 사용)
# 예측한 숫자를 다시 문자로 바꾸는 과정
# 인덱스 숫자번호 : 문자 로 만드는 방법 -> key로는 추적이 안되기 때문에 필요
reverse_input_token_index = {idx:char for char,idx in input_token_index.items()}
reverse_target_token_index = {idx:char for char,idx in target_token_index.items()}

# encoder_input_data = 3D배열 (샘플, 시퀀스 길이, 문자 사전 크기)
# Encoder 입력용 -> 각 시퀀스의 문자를 원-핫 벡터로 표현
encoder_input_data = np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),
                              dtype = 'float32')
# Decoder 입력용-> 매 시점(t)에 이전 실제 출력(정답) 문자 토큰을 넣어 모델이 학습하도록 함
decoder_input_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),
                              dtype = 'float32')
# Decoder 정답용 -> 한 타임스텝 뒤로 밀린 정답
                    # 예: decoder_input_data: \t b o n j o u r
                    # decoder_target_data: b o n j o u r \n
                    # LSTM이 다음 시점 출력(y_t)과 비교하며 학습
decoder_target_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),
                              dtype = 'float32')

# 문자별로 원핫 인코딩
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    # 1) encoder 입력
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0 # (샘플, 시퀀스 길이, 입력 문자 수)/ i번째 샘플, t번재 시점에서 문자 char를 원핫인코딩
    # 2) Decoder 입력 & 정답
    for t, char in enumerate(target_text):
       #'decoder_input_data: 전체 타겟 시퀀스 (시작 토큰 포함)'
        decoder_input_data[i, t, target_token_index[char]] = 1.0  # (샘플, 시퀀스 길이, 출력 문자 수)/ target_text 전체 문장을 t 시점마다 원-핫 인코딩

       #'decoder_target_data: 한 타임스텝 앞선 정답 (Teacher Forcing용)'
        if t > 0:   # 첫번째 입력을 제외하고 
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0   # 시점 개념이 아닌 타겟의 관점에서 input보다 하나 짧은거 가져와! 그래서 t-1인 거 같은 t가 시퀀스 길이니까
        # = 1.0  이라는 것은 원핫인코딩에서 가면 1 나다라는 0으로 하는 것처럼, 이 지점이 1이야 라는 의미
print(f'고유 입력 문자수: {num_encoder_tokens}')
print(f'고유타겟 입력 문자수 : {num_decoder_tokens}')
print(f'최대 입력 문자길이 : {max_encoder_seq_length}')
print(f'최대 타겟 문자길이 : {max_decoder_seq_length}')
print(f'# 샘플 시퀀스길이, 문자 사전 크기')
print(f'endocer_input_sdate :{encoder_input_data.shape}')
print(f'decoder_input_data.shap : {decoder_input_data.shape}')
print(f'decoder_target_data.shape : {decoder_target_data.shape}')


고유 입력 문자수: 19
고유타겟 입력 문자수 : 22
최대 입력 문자길이 : 12
최대 타겟 문자길이 : 20
# 샘플 시퀀스길이, 문자 사전 크기
endocer_input_sdate :(4, 12, 19)
decoder_input_data.shap : (4, 20, 22)
decoder_target_data.shape : (4, 20, 22)


<span style="color: Gold"> 3. LSTM 기반 Seq2Seq 인코더-디코더 학습 모델 구축

 개념:    
    Encoder: 입력 시퀀스를 처리하고 최종 상태(h, c) 출력  
    Decoder: Encoder 상태를 초기값으로 받아 타겟 시퀀스 생성  
    return_state=True: LSTM 내부 상태(h, c) 반환  
    return_sequences=True: 모든 타임스텝 출력  

설명:
    encoder_states: [h, c] (hidden state, cell state)  
    decoder_lstm: 초기 상태로 encoder_states 전달  
    decoder_dense: Softmax로 각 타임스텝의 문자 확률 분포 생성  

<span style="font-size:12px;">

<span style="color: lightblue;"> LSTM 구조

- LSTM(Long Short-Term Memory)은 RNN 계열의 한 종류로, 시퀀스를 처리하면서 장기 의존성을 기억할 수 있도록 설계된 구조  
<br>
- Hidden state (h_t)
    - 현재 시점(t)에서 LSTM이 계산한 출력 벡터
    - 다음 시점으로 전달되며, 동시에 외부에서 바로 읽어서 사용할 수도 있음

- Cell state (c_t)
    - 시퀀스 전체의 장기 기억(Long-term memory) 역할
    - 정보를 선택적으로 기억하거나 잊게 함 (forget gate, input gate, output gate가 제어)

In [49]:

latent_dim = 256  # LSTM 은닉 차원 (내부 표현 크기)
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

# ==================== Encoder ====================
# 인풋레이어 생성
# encoder_inputs: (배치 크기, 시퀀스 길이, 입력 문자 수)
encoder_inputs = Input(shape=(None, num_encoder_tokens), name='encoder_input')
encoder_lstm = LSTM(latent_dim, return_state=True, name='encoder_lstm')
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# encoder_outputs는 사용하지 않고, 내부 상태(state_h, state_c)만 디코더로 전달
# 입력 시퀀스를 LSTM에 통과시켜서 마지막 은닉상태(state_h)와 셀상태(state_c)를 받아서
# 두 상태는 입력 문장의 의미(context)를 압축한 벡터
encoder_states = [state_h, state_c]

# ==================== Decoder ====================
decoder_inputs = Input(shape=(None, num_decoder_tokens), name='decoder_input')
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')

# 디코더 초기 상태로 인코더 최종 상태 사용 (컨텍스트 전달)
# 인코더의 상태 (state_h, state_c)를 초기상태로 받아서 자신의 입력 decoder_inputs 을 기반으로
# 다음단어를 예측 --> 각 시점의 출력은 Dense+softmax를 거쳐서 단어(문자) 확률분포
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# 각 타임스텝에서 문자 확률 분포 생성
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

# ==================== 학습 모델 ====================
model = Model([encoder_inputs, decoder_inputs], decoder_outputs, name='seq2seq_training')

print("\n 모델 구조:")
model.summary()


 모델 구조:


Model: "seq2seq_training"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input       │ (None, None, 19)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input       │ (None, None, 22)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm (LSTM) │ [(None, 256),     │    282,624 │ encoder_input[0]… │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │    285,696 │ decoder_input[0]… │
│                     │ 256), (None,      │            │ encoder_lstm[0][… │
│                     │ 256), (None,      │            │ encoder_lstm[0][… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, None, 22)  │      5,654 │ decoder_lstm[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 573,974 (2.19 MB)

 Trainable params: 573,974 (2.19 MB)

 Non-trainable params: 0 (0.00 B)

 <span style="color: Gold"> 4. 목적: Seq2Seq 모델 컴파일 및 학습 실행   

 개념:  
    - categorical_crossentropy: 다중 클래스(문자 사전) 손실  
    - Teacher Forcing: decoder_input_data는 정답 시퀀스 전체 제공  
    - 학습 목표: decoder_target_data (한 타임스텝 앞당긴 정답)  

 설명:  
    - optimizer='rmsprop': 순환신경망에 안정적인 최적화 알고리즘  
    - epochs=100: 작은 데이터셋이므로 충분한 반복 필요  
    - batch_size=2: 메모리 효율 (실제로는 전체 4개 샘플 사용)  

In [56]:
model.compile(
    optimizer = 'rmsprop',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    [encoder_input_data,decoder_input_data],  # seq2seq
    decoder_target_data,
    batch_size = 2,
    epochs = 500,
    # vallidation_split = 0.0,  # 데이터셋이 작아서 분할 안함
    verbose = 0 # 0 출력안하고 1은 간단하게 2 좀더 출
)

In [67]:
history.history['accuracy'][-1]
# model.save_weigts('seq2seq_weight.h5')

0.48750001192092896

<span style="color: Gold">  5. 학습된 가중치를 사용해 실제 번역용 추론 모델 구축

핵심 개념:  
Encoder 모델: 입력 → 내부 상태 추출  
Decoder 모델: 이전 상태 + 현재 입력 → 다음 문자 예측  
추론 시에는 Teacher Forcing 없이 자기 예측을 다음 입력으로 사용  

설명:    
encoder_model: 입력 문장 → [h, c] 상태 출력  
decoder_model: 한 타임스텝씩 반복 실행  
각 스텝에서 가장 높은 확률의 문자 선택 (Greedy Decoding)  

In [ ]:
# encoder 추론모델
encoder_model = Model(encoder_inputs, encoder_states, name = 'encoder_inference')

In [59]:
# decoder 추론모델
# 이전 타임스텝의 상태를 입력으로 받음
decoder_state_input_h = Input(shape=(latent_dim,), name = 'decoder_state_h')
decoder_state_input_c = Input(shape=(latent_dim,), name = 'decoder_state_c')
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]
# LSTM실행(이전상태 + 현재입력)
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_state_inputs)
decoder_states = [state_h, state_c]
# 문자 확률 분포 생성
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_state_inputs,
    [decoder_outputs] + decoder_states,
    name = 'decoder_inference'
)

<span style="color: Gold">  6. 입력 문장을 번역하는 디코딩 함수 구현 및 테스트

 개념:  
    - Greedy Decoding: 매 스텝 가장 높은 확률 문자 선택  
    - 종료 조건: '\n' 토큰 생성 또는 최대 길이 도달  
    - 자기회귀적 생성: 이전 예측을 다음 입력으로 반복 사용  

 설명:  
    1. Encoder로 입력 문장의 상태 벡터 추출  
    2. 시작 토큰('\t')으로 Decoder 시작  
    3. 반복: 현재 문자 예측 → 다음 입력으로 사용  
    4. '\n' 만나면 종료  

In [63]:
def decode_sequence(input_seq):
    """
    입력 시퀀스(원-핫 인코딩)를 받아 번역된 문자열 반환
    """
    # 1단계: Encoder로 상태 벡터 추출
    states_value = encoder_model.predict(input_seq, verbose=0)
    
    # 2단계: 디코더 시작 토큰 준비 ('\t')
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.0
    
    # 3단계: 문자를 하나씩 생성
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        # 현재 문자 예측 + 다음 상태 업데이트
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )
        
        # 가장 높은 확률의 문자 선택 (Greedy)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_token_index[sampled_token_index]
        
        # 문자 추가
        decoded_sentence += sampled_char
        
        # 종료 조건 체크
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True
        
        # 다음 스텝 준비: 현재 예측을 다음 입력으로
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        
        # 상태 업데이트
        states_value = [h, c]
    
    return decoded_sentence

In [64]:

for seq_index in range(len(input_texts)):
    # 원핫인코딩 입력 추출
    input_seq =  encoder_input_data[seq_index:seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    
    # 시작/종료 토큰 제거
    decoded_sentence = decoded_sentence.replace('\t','').replace('\n','')
    print(f'입력문장 : {input_texts[seq_index]}')
    print(f'정답문장 : {target_texts[seq_index][1:-1]}')  # 시작 종료토큰제거
    print(f'모델 예측 : {decoded_sentence}')
    print('-'*100) 

입력문장 : Hello
정답문장 : Bonjour
모델 예측 : Bonjour
----------------------------------------------------------------------------------------------------
입력문장 : How are you
정답문장 : Comment allez-vous
모델 예측 : Comment allllz
----------------------------------------------------------------------------------------------------
입력문장 : Good morning
정답문장 : Bonjour matin
모델 예측 : Bonjour maaii
----------------------------------------------------------------------------------------------------
입력문장 : Thank you
정답문장 : Merci
모델 예측 : Merci
----------------------------------------------------------------------------------------------------
